# 6. Transformations d'ensembles de données

scikit-learn fournit une bibliothèque de transformateurs, qui peuvent nettoyer (voir [6.3. Prétraitement des données](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)), réduire (voir [6.5. Réduction de dimensionnalité non supervisée](https://scikit-learn.org/stable/modules/unsupervised_reduction.html#data-reduction)), étendre (voir [6.7. Approximation du noyau](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation)) ou générer (voir [6.2. Extraction de caractéristiques](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-extraction)) des représentations d'entités.

Comme d'autres estimateurs, ceux-ci sont représentés par des classes avec une méthode d'ajustement `fit`, qui apprend les paramètres du modèle (par exemple, la moyenne et l'écart type pour la normalisation) à partir d'un ensemble d'apprentissage, et une méthode de transformation `transform` qui applique ce modèle de transformation à de nouvelles données. `fit_transform` peut être plus pratique et efficace pour modéliser et transformer simultanément les données d'apprentissage.

La combinaison de tels transformateurs, en parallèle ou en série, est traitée dans [6.1. Pipelines et estimateurs composites](https://scikit-learn.org/stable/modules/compose.html#combining-estimators). [6.8. Les métriques par paires, es Affinités et les Noyaux](https://scikit-learn.org/stable/modules/metrics.html#metrics) couvrent la transformation des espaces de caractéristiques en matrices d'affinité, tandis que la [6.9. transformation de la cible de prédiction (y)](https://scikit-learn.org/stable/modules/preprocessing_targets.html#preprocessing-targets) considère les transformations de l'espace cible (par exemple, les étiquettes catégorielles) à utiliser dans scikit-learn.

6.1. Pipelines et estimateurs composites
* 6.1.1. Pipeline : estimateurs de chaînage
* 6.1.2. Transformer la cible en régression
* 6.1.3. FeatureUnion : espaces d'entités composites
* 6.1.4. ColumnTransformer pour les données hétérogènes
* 6.1.5. Visualisation des estimateurs composites

6.2. Extraction de caractéristiques
* 6.2.1. Chargement de fonctionnalités à partir de dicts
* 6.2.2. Hachage des fonctionnalités
* 6.2.3. Extraction de caractéristiques de texte
* 6.2.4. Extraction de caractéristiques d'image

6.3. Prétraitement des données
* 6.3.1. Standardisation, ou suppression de la moyenne et mise à l'échelle de la variance
* 6.3.2. Transformation non linéaire
* 6.3.3. Normalisation
* 6.3.4. Encodage des caractéristiques catégorielles
* 6.3.5. Discrétisation
* 6.3.6. Imputation des valeurs manquantes
* 6.3.7. Génération de caractéristiques polynomiales
* 6.3.8. Transformateurs personnalisés

6.4. Imputation des valeurs manquantes
* 6.4.1. Imputation univariée vs imputation multivariée
* 6.4.2. Imputation de caractéristique univariée
* 6.4.3. Imputation de caractéristiques multivariées
* 6.4.4. Références
* 6.4.5. Imputation des plus proches voisins
* 6.4.6. Marquage des valeurs imputées
* 6.4.7. Estimateurs qui gèrent les valeurs NaN

6.5. Réduction de dimensionnalité non supervisée
* 6.5.1. ACP : analyse en composantes principales
* 6.5.2. Projections aléatoires
* 6.5.3. Agglomération de fonctionnalités

6.6. Projection aléatoire
* 6.6.1. Le lemme de Johnson-Lindenstrauss
* 6.6.2. Projection aléatoire gaussienne
* 6.6.3. Projection aléatoire clairsemée
* 6.6.4. Transformation inverse

6.7. Approximation du noyau
* 6.7.1. Méthode Nystroem pour l'approximation du noyau
* 6.7.2. Noyau de fonction de base radiale
* 6.7.3. Additif Chi Squared Kernel
* 6.7.4. Noyau au carré de chi asymétrique
* 6.7.5. Approximation du noyau polynomial via Tensor Sketch
* 6.7.6. Détails mathématiques

6.8. Métriques par paires, affinités et noyaux
* 6.8.1. Similitude cosinus
* 6.8.2. Noyau linéaire
* 6.8.3. Noyau polynomial
* 6.8.4. Noyau sigmoïde
* 6.8.5. Noyau RBF
* 6.8.6. Noyau laplacien
* 6.8.7. Noyau du chi carré

6.9. Transformer la cible de prédiction (y)
* 6.9.1. Binarisation des étiquettes
* 6.9.2. Encodage des étiquettes

# 6.1. Pipelines et estimateurs composites

Les transformateurs sont généralement combinés avec des classifieurs, des régresseurs ou d'autres estimateurs pour construire un estimateur composite. L'outil le plus courant est un [6.1.1. Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators). Pipeline est souvent utilisé en combinaison avec [6.1.3. FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#feature-union) qui concatène la sortie des transformateurs dans un espace de caractéristiques composite. [6.1.2. TransformedTargetRegressor](https://scikit-learn.org/stable/modules/compose.html#transformed-target-regressor) traite de la transformation de la cible (c'est-à-dire log-transform y). En revanche, Pipelines ne transforme que les données observées (X).

## 6.1.1. Pipeline : estimateurs de chaînage¶

[`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) peut être utilisé pour enchaîner plusieurs estimateurs en un seul. Ceci est utile car il y a souvent une séquence fixe d'étapes dans le traitement des données, par exemple la sélection des caractéristiques, la normalisation et la classification. Pipeline a plusieurs objectifs ici :

**Commodité et encapsulation**

Il suffit d'appeler `fit` et `predict` une fois sur vos données pour ajuster toute une séquence d'estimateurs.

**Sélection des paramètres communs**

Vous pouvez effectuer une recherche par grille sur les paramètres de tous les estimateurs du pipeline à la fois.

**Sécurité**

Les pipelines permettent d'éviter les fuites de statistiques de vos données de test dans le modèle formé lors de la validation croisée, en garantissant que les mêmes échantillons sont utilisés pour former les transformateurs et les prédicteurs.

Tous les estimateurs d'un pipeline, à l'exception du dernier, doivent être des transformateurs (c'est-à-dire qu'ils doivent avoir une méthode `transform`). Le dernier estimateur peut être de n'importe quel type (transformateur, classificateur, etc.).

### Usage

#### Construction

Le [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) est construit à l'aide d'une liste de paires `(key, value)`, où `key` est une chaîne contenant le nom que vous souhaitez donner à cette étape et `value` est un objet estimateur :

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe
# Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

#### Accéder aux étapes

Les estimateurs d'un pipeline sont stockés sous forme de liste dans l'attribut `steps`, mais sont accessibles par index ou nom en indexant (avec `[idx]`) le Pipeline :

In [2]:
pipe.steps[0]
# ('reduce_dim', PCA())
pipe[0]
# PCA()
pipe['reduce_dim']
# PCA()

PCA()

L'attribut `named_steps` de Pipeline permet d'accéder aux étapes par leur nom avec complétion par tabulation dans des environnements interactifs :

In [3]:
pipe.named_steps.reduce_dim is pipe['reduce_dim']

True

Un sous-pipeline peut également être extrait en utilisant la notation de découpage couramment utilisée pour les séquences Python telles que les listes ou les chaînes (bien que seul un pas de 1 soit autorisé). Ceci est pratique pour n'effectuer qu'une partie des transformations (ou leur inverse) :

In [4]:
pipe[:1]
# Pipeline(steps=[('reduce_dim', PCA())])
pipe[-1:]
# Pipeline(steps=[('clf', SVC())])

Pipeline(steps=[('clf', SVC())])

#### Paramètres imbriqués

Les paramètres des estimateurs du pipeline sont accessibles à l'aide de la syntaxe `<estimator>__<parameter>` :

In [5]:
pipe.set_params(clf__C=10)
# Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC(C=10))])

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC(C=10))])

Ceci est particulièrement important pour effectuer des recherches sur grille :

In [6]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(reduce_dim__n_components=[2, 5, 10],
                  clf__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Les étapes individuelles peuvent également être remplacées en tant que paramètres, et les étapes non finales peuvent être ignorées en les définissant sur `'passthrough'` :

In [7]:
from sklearn.linear_model import LogisticRegression
param_grid = dict(reduce_dim=['passthrough', PCA(5), PCA(10)],
                  clf=[SVC(), LogisticRegression()],
                  clf__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Les estimateurs du pipeline peuvent être récupérés par index :

In [8]:
pipe[0]

PCA()

ou par nom :

In [9]:
pipe['reduce_dim']

PCA()

Pour activer l'inspection du modèle, [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) a une méthode `get_feature_names_out()`, comme tous les transformateurs. Vous pouvez utiliser le découpage de pipeline pour obtenir les noms de fonctionnalités à chaque étape :

In [10]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
iris = load_iris()
pipe = Pipeline(steps=[
    ('select', SelectKBest(k=2)),
    ('clf', LogisticRegression())])
pipe.fit(iris.data, iris.target)
# Pipeline(steps=[('select', SelectKBest(...)), ('clf', LogisticRegression(...))])
pipe[:-1].get_feature_names_out()

array(['x2', 'x3'], dtype=object)

### Voir aussi

[3.2. Estimateurs composites et espaces de paramètres](https://scikit-learn.org/stable/modules/grid_search.html#composite-grid-search)

### Exemple - [Pipeline ANOVA SVM](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection_pipeline.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-pipeline-py)

Cet exemple montre comment une sélection de caractéristiques peut être facilement intégrée dans un pipeline d'apprentissage automatique.

Nous montrons également que vous pouvez facilement introspecter une partie du pipeline.

Nous allons commencer par générer un jeu de données de classification binaire. Par la suite, nous diviserons le jeu de données en deux sous-ensembles.

In [11]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=20,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    n_clusters_per_class=2,
    random_state=42,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Une erreur courante commise avec la sélection de caractéristiques consiste à rechercher un sous-ensemble de caractéristiques discriminantes sur l'ensemble de données complet au lieu d'utiliser uniquement l'ensemble d'apprentissage. L'utilisation du [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) scikit-learn empêche de commettre une telle erreur.

Ici, nous allons montrer comment créer un pipeline où la première étape sera la sélection des caractéristiques.

Lors de l'appel de `fit` sur les données d'entraînement, un sous-ensemble de caractéristiques sera sélectionné et l'index de ces caractéristiques sélectionnées sera stocké. Le sélecteur de caractéristiques réduira ensuite le nombre de caractéristiques et transmettra ce sous-ensemble au classifieur qui sera formé.

In [12]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

anova_filter = SelectKBest(f_classif, k=3)
clf = LinearSVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)

Pipeline(steps=[('selectkbest', SelectKBest(k=3)), ('linearsvc', LinearSVC())])

Une fois l'entraînement accompli, nous pouvons prédire sur de nouveaux échantillons inédits. Dans ce cas, le sélecteur de caractéristiques sélectionnera uniquement les caractéristiques les plus discriminantes en fonction des informations stockées pendant l'entraînement. Ensuite, les données seront transmises au classifieur qui fera la prédiction.

Ici, nous rapportons les mesures finales via un rapport de classification.

In [13]:
from sklearn.metrics import classification_report

y_pred = anova_svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.80      0.86        15
           1       0.75      0.90      0.82        10

    accuracy                           0.84        25
   macro avg       0.84      0.85      0.84        25
weighted avg       0.85      0.84      0.84        25



Sachez que vous pouvez inspecter une étape du pipeline. Par exemple, nous pourrions être intéressés par les paramètres du classifieur. Puisque nous avons sélectionné trois caractéristiques, nous nous attendons à avoir trois coefficients.

In [14]:
anova_svm[-1].coef_

array([[0.75791019, 0.27158925, 0.26109575]])

Cependant, nous ne savons pas quelles caractéristiques ont été sélectionnées dans l'ensemble de données d'origine. On pourrait procéder de plusieurs manières. Ici, nous allons inverser la transformation de ces coefficients pour obtenir des informations sur l'espace d'origine.

In [15]:
anova_svm[:-1].inverse_transform(anova_svm[-1].coef_)

array([[0.        , 0.        , 0.75791019, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.27158925,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.26109575]])

Nous pouvons voir que les trois premières caractéristiques étaient les caractéristiques sélectionnées par la première étape.

### Exemple - ...